In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import kw

In [ ]:
dfs = kw.getdf_xlsx()

df_master = dfs['試験マスタ']
df_data = dfs['試験データ']

df =df_master.merge(df_data, on='test_id')

df['theoretical_power'] = df['capacity'] * df['total_head'] * 9.8 / 60
df['pump_eff'] = df['theoretical_power'] / df['shaft_power'] * 100

# データを'model'と'test_no'でグループ化し、それぞれのグループに対してグラフを作成します
models = df['model'].unique()
models

In [ ]:
# 軸の設定
x_axis = 'capacity'
X_axis_unit = 'm3/min'
xlim_min = 0
xlim_max = None

y_axis1 = 'total_head'
y_axis_unit1 = 'm'

y_axis2 = 'pump_eff'
y_axis_unit2 = '%'

y_axis3 = 'shaft_power'
y_axis_unit3 = 'kW'


## 暫定

In [ ]:
#点と近似曲線を両方描画するようにする

# データを'model'と'test_no'でグループ化し、それぞれのグループに対してグラフを作成します
all_models = df['model'].unique()

selected_models = [
   ['AZ65F3.7', 'AZ50F3.7', 'AZ40F3.7'],
   'AZ50F2.2',
   'AZ40F2.2'
]

# selected_modelsの各要素がリストでなければ、リストに変換します
selected_models = [model if isinstance(model, list) else [model] for model in selected_models]

# モデル毎の色を指定します
model_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']

# 軸毎の色を指定します
axis_colors = ['tab:blue', 'tab:orange', 'tab:green']  # Noneを設定します

# model_colorsがNoneの場合、axis_colorsで実行
if model_colors is None:
    model_colors = axis_colors

# axis_colorsがNoneまたは両方ある場合、model_colorsで実行
if axis_colors is None or model_colors is not None:
    axis_colors = model_colors

# リストの長さが不足する場合の対策として、リストを複数回繰り返す
model_colors = model_colors * max(sum(len(models) for models in selected_models) // len(model_colors) + 1, 1)
axis_colors = axis_colors * max(3 // len(axis_colors) + 1, 1)

# サブプロットの行数を設定します
num_rows = 2

# モデルの数に基づいてサブプロットの列数を計算します
num_cols = len(selected_models) // num_rows
if len(selected_models) % num_rows != 0:  # モデルの数が行数で割り切れない場合
    num_cols += 1  # 列数を1つ増やします

# Figure and array of axes are created
fig, axs = plt.subplots(num_rows, num_cols, figsize=(8 * num_cols, 6 * num_rows))
axs = axs.ravel()

color_index = 0
for i, models in enumerate(selected_models):
    for model in models:
        color = model_colors[color_index]  # model_colors listから色を選択
        color_index += 1

        df_model = df[df['model'] == model]

        # Current axes are selected
        ax1 = axs[i]

        ax1.set_xlabel(f'{x_axis} ({X_axis_unit})')
        ax1.set_ylabel(f'{y_axis1} ({y_axis_unit1})', color=axis_colors[0])
        # (line1,) = ax1.plot(df_model[x_axis], df_model[y_axis1], color=color, label=y_axis1 + " (" + model + ")")  # colorをmodel_colorsから選択
        
        # 近似曲線
        # coeffs1 = np.polyfit(df_model[x_axis], df_model[y_axis1], 6)
        # approx_y_axis1 = np.polyval(coeffs1, df_model[x_axis])
        # (line1,) = ax1.plot(df_model[x_axis], approx_y_axis1, color=color, label=y_axis1 + " (" + model + ")")  
        
        # スプライン近似
        spline1 = UnivariateSpline(df_model[x_axis], df_model[y_axis1])
        approx_y_axis1 = spline1(df_model[x_axis])
        (line1,) = ax1.plot(df_model[x_axis], approx_y_axis1, color=color, label=y_axis1 + " (" + model + ")") 
        
        ax1.tick_params(axis='y', labelcolor=axis_colors[0])
        ax1.set_xlim(0,)
        ax1.set_ylim(0, 100)
        ax1.grid(True)

        # 2つ目のy軸の作成
        ax2 = ax1.twinx()

        ax2.spines['left'].set_position(('axes', -0.1))
        ax2.yaxis.set_label_position('left')
        ax2.yaxis.set_ticks_position('left')

        ax2.set_ylabel(f'{y_axis2} ({y_axis_unit2})', color=axis_colors[1])
        # (line2,) = ax2.plot(df_model[x_axis], df_model[y_axis2], color=color, label=y_axis2 + " (" + model + ")")  # colorをmodel_colorsから選択
        
        # 近似曲線
        # coeffs2 = np.polyfit(df_model[x_axis], df_model[y_axis2], 3)
        # approx_y_axis2 = np.polyval(coeffs2, df_model[x_axis])
        # (line2,) = ax2.plot(df_model[x_axis], approx_y_axis2, color=color, label=y_axis2 + " (" + model + ")")  
        
        # スプライン近似
        spline2 = UnivariateSpline(df_model[x_axis], df_model[y_axis2])
        approx_y_axis2 = spline2(df_model[x_axis])
        (line2,) = ax2.plot(df_model[x_axis], approx_y_axis2, color=color, label=y_axis2 + " (" + model + ")") 
        
        ax2.tick_params(axis='y', labelcolor=axis_colors[1])
        ax2.set_ylim(0, 100)  # total_headの範囲を設定

        # 3つ目のy軸の作成
        ax3 = ax1.twinx()

        ax3.spines['left'].set_position(('axes', -0.2))
        ax3.yaxis.set_label_position('left')
        ax3.yaxis.set_ticks_position('left')

        ax3.set_ylabel(f'{y_axis3} ({y_axis_unit3})', color=axis_colors[2])
        # (line3,) = ax3.plot(df_model[x_axis], df_model[y_axis3], color=color, label=y_axis3 + " (" + model + ")")  # colorをmodel_colorsから選択
        
        # 近似曲線
        coeffs3 = np.polyfit(df_model[x_axis], df_model[y_axis3], 3)
        approx_y_axis3 = np.polyval(coeffs3, df_model[x_axis])
        (line3,) = ax3.plot(df_model[x_axis], approx_y_axis3, color=color, label=y_axis2 + " (" + model + ")")  
        
        ax3.tick_params(axis='y', labelcolor=axis_colors[2])
        ax3.set_ylim(0, 10) 

    # 凡例の作成
    lines = [line1, line2, line3]
    # labels = [l.get_label() for l in lines]
    
    labels = models
    # ax1.legend(lines, labels, loc='best')
    ax1.legend(labels, loc='best')

    ax1.set_title(f'Models: {", ".join(models)}')

# 使用しないサブプロットを削除
for j in range(i+1, num_rows*num_cols):
    fig.delaxes(axs[j])

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


In [ ]:
df_model[x_axis]

## 関数

In [ ]:
def setup_axis(ax, position, x, y, color, label, y_labelcolor, ylim):
    """
    各軸を設定する関数
    
    Parameters
    ----------
    ax : Axes
        設定を行う軸
    position : float
        軸の位置
    x : str
        x軸のデータ名
    y : str
        y軸のデータ名
    color : str
        線の色
    label : str
        凡例のラベル名
    y_labelcolor : str
        y軸の単位
    ylim : tuple
        y軸の範囲
    Returns
    -------
    Line2D
        プロットされた線のオブジェクト
    """
    ax.spines['left'].set_position(('axes', position))  # 軸の位置を設定
    ax.yaxis.set_label_position('left')  # ラベルの位置を設定
    ax.yaxis.set_ticks_position('left')  # ティックの位置を設定

    ax.set_ylabel(f'{y} ({y_labelcolor})', color=color)  # y軸のラベルを設定
    line, = ax.plot(df_model[x], df_model[y], color=color, label=label)  # データをプロット
    ax.tick_params(axis='y', labelcolor=color)  # y軸のティックの色を設定
    ax.set_ylim(*ylim)  # y軸の範囲を設定

    return line

In [ ]:
# データを'model'と'test_no'でグループ化し、それぞれのグループに対してグラフを作成します
models = df['model'].unique()

# サブプロットの行数を設定します
num_rows = 2

# モデルの数に基づいてサブプロットの列数を計算します
num_cols = len(models) // num_rows
if len(models) % num_rows != 0:  # モデルの数が行数で割り切れない場合
    num_cols += 1  # 列数を1つ増やします

# Figure and array of axes are created
fig, axs = plt.subplots(num_rows, num_cols, figsize=(8 * num_cols, 6 * num_rows))

axs = axs.ravel() 
for i, model in enumerate(models):
    df_model = df[df['model'] == model]
    ax1 = axs[i]

    ax1.set_xlabel(f'{x_axis} ({X_axis_unit})')  # x軸のラベルを設定
    # ax1.set_xlim(0, None)  # x軸の範囲を設定
    ax1.grid(True)  # グリッドを表示

    # 各y軸のパラメータを設定
    y_params = [
        (ax1, 0, x_axis, y_axis1, 'tab:red', y_axis1, y_axis_unit1, (0, 100)),
        (ax1.twinx(), -0.1, x_axis, y_axis2, 'tab:blue', y_axis2, y_axis_unit2, (0, 100)),
        (ax1.twinx(), -0.2, x_axis, y_axis3, 'tab:blue', y_axis3, y_axis_unit3, (0, 10))
    ]

    lines = [setup_axis(*params) for params in y_params]  # 各軸を設定し、プロットした線のリストを作成
    labels = [l.get_label() for l in lines]  # 各線のラベルをリスト化
    ax1.legend(lines, labels, loc='best')  # 凡例を作成
    ax1.set_title(f'Model: {model}')  # グラフのタイトルを設定


# 使用しないサブプロットを削除
for j in range(i+1, num_rows*num_cols):
    fig.delaxes(axs[j])

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()
